<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150 align="right">
<img src=images/Acme.jpeg width=100 align="left">


Jenkins Integration
---
## Business Values
 * Provide a way for operators to manage policy without being involved in generating tokens or creds for applications (minimize coordination) 
 * Operators have less work to do
 * Orchestration tools can take over the task of giving applications secrets access

## How AppRole Auth Method works

How can a Jenkins server programmatically request a token so that it can read secrets from Vault?

Using the AppRole which is an authentication mechanism within Vault to allow machines or apps to acquire a token to interact with Vault and using the policies you can set access limitations for your app.
<br />   
<img src="images/vault-jenkins-workflow.png" width=800>


### Setup
---

In [3]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [4]:
vault login root

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       dRq6usE3NFp7vjYtgtH8uqQv
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


In [ ]:
vault status; vault version

### Demo
---

### Enable the approle authentication method


In [3]:
vault auth enable approle

Error enabling approle auth: Error making API request.

URL: POST http://127.0.0.1:8200/v1/sys/auth/approle
Code: 400. Errors:

* path is already in use at approle/


### Create a secret
NOTE: num_uses determines how many times secret can be used.

In this example, a SecretID of the myapp role can be used for up to 5 times to authenticate and fetch a client token. After the number of uses is reached, the SecretID expires and you would need to generate a new one. This is similar to forcing a password rotation.

In [14]:
# use KV 1

tee /tmp/orchestrator-policy.hcl <<"EOF"
  path "secret1/*" {
    capabilities = [ "read", "list"]
  }

  path "aws/*" {
    capabilities = [ "read" ]
  }

  path "mdb1/*" {
    capabilities = [ "read" ]
  }
EOF

  path "secret1/*" {
    capabilities = [ "read", "list"]
  }

  path "aws/*" {
    capabilities = [ "read" ]
  }

  path "mdb1/*" {
    capabilities = [ "read" ]
  }


In [15]:
vault policy write orchestrator-policy /tmp/orchestrator-policy.hcl

Success! Uploaded policy: orchestrator-policy


### Create a role id for Jenkins
Typically a constant value, provide this to the app developer team

In [6]:
vault write auth/approle/role/orchestrator token_max_ttl=4h token_ttl=30m token_policies="orchestrator-policy"

Success! Data written to: auth/approle/role/orchestrator


In [7]:
vault read auth/approle/role/orchestrator/role-id

Key        Value
---        -----
role_id    00fdf1d9-8ff0-4df4-d113-ae3e3f8ed321


**Generate a Secret ID**

In [8]:
vault write -f auth/approle/role/orchestrator/secret-id

Key                   Value
---                   -----
secret_id             40f74c8d-b0a3-bb63-8a9e-e9e83e7194bb
secret_id_accessor    bec41b91-47a7-a250-aead-a2252b1bf4f6


### Use API

In [10]:
export APPROLE="orchestrator"
export VAULT_TOKEN=$(vault print token)
ROLE_ID=$(curl -s -H "X-Vault-Token: ${VAULT_TOKEN}" "http://127.0.0.1:8200/v1/auth/approle/role/${APPROLE}/role-id" | jq -r '.data.role_id')
echo $ROLE_ID

00fdf1d9-8ff0-4df4-d113-ae3e3f8ed321


In [10]:
#another secret-id
vault write -f auth/approle/role/orchestrator/secret-id

Key                   Value
---                   -----
secret_id             1424f553-5412-24eb-044f-5369e96cb1e7
secret_id_accessor    86822756-4b86-6c3c-a21b-66a8133cf9f5


**Using CURL instead**  

In [ ]:
SECRET_ID=$(curl -s -X POST -H "X-Vault-Token:${VAULT_TOKEN}" "http://127.0.0.1:8200/v1/auth/jenkins/role/${APPROLE}/secret-id" | jq -r '.data.secret_id')
echo ${SECRET_ID}

### Get a token for logging into Vault using roleID and secretID

In [11]:
ROLE_ID=00fdf1d9-8ff0-4df4-d113-ae3e3f8ed321 SECRET_ID=40f74c8d-b0a3-bb63-8a9e-e9e83e7194bb;
vault write auth/approle/login role_id=${ROLE_ID} secret_id=${SECRET_ID}

Key                     Value
---                     -----
token                   s.WFxlF06y382aTScsl3ZT3AJE
token_accessor          GEfIvhEFtVbyFss9ScdNo1md
token_duration          30m
token_renewable         true
token_policies          ["default" "orchestrator-policy"]
identity_policies       []
policies                ["default" "orchestrator-policy"]
token_meta_role_name    orchestrator


**Again, we can do the same using CURL**

In [ ]:
APP_ROLE_TOKEN=$(curl -s -X POST -d '{"role_id": "'"${ROLE_ID}"'", "secret_id": "'"${SECRET_ID}"'"}' http://127.0.0.1:8200/v1/auth/approle/login | jq -r '.auth.client_token')
echo ${APP_ROLE_TOKEN}

In [16]:
APP_ROLE_TOKEN=s.mSaXjM2gjtQrbh2h1sRdMktF
vault login ${APP_ROLE_TOKEN}

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                     Value
---                     -----
token                   s.mSaXjM2gjtQrbh2h1sRdMktF
token_accessor          FTgfgFExmtv429q2eO2CM06K
token_duration          19m40s
token_renewable         true
token_policies          ["default" "jenkins-policy"]
identity_policies       []
policies                ["default" "jenkins-policy"]
token_meta_role_name    jenkins


In [23]:
VAULT_TOKEN=root vault list auth/token/accessors

Keys
----
AlupWSwLEGqrVAuBtxqC0ymu
FTgfgFExmtv429q2eO2CM06K
SRRjFv6GWrxQzPHtyDupBRlQ
WhqpAsK30bQgNj9LcttppqRZ
vMBmDMzQcFBsc1oCuhFDqT7A


In [32]:
VAULT_TOKEN=root vault token lookup -accessor  FTgfgFExmtv429q2eO2CM06K

Key                 Value
---                 -----
accessor            FTgfgFExmtv429q2eO2CM06K
creation_time       1615646146
creation_ttl        30m
display_name        approle
entity_id           df55bd26-ff52-c59d-2e8d-a7ff154d448a
expire_time         2021-03-13T23:05:46.589912+08:00
explicit_max_ttl    0s
id                  n/a
issue_time          2021-03-13T22:35:46.589915+08:00
meta                map[role_name:jenkins]
num_uses            0
orphan              true
path                auth/approle/login
policies            [default jenkins-policy]
renewable           true
ttl                 11m4s
type                service


In [37]:
VAULT_TOKEN=s.mSaXjM2gjtQrbh2h1sRdMktF vault token lookup

Key                 Value
---                 -----
accessor            FTgfgFExmtv429q2eO2CM06K
creation_time       1615646146
creation_ttl        30m
display_name        approle
entity_id           df55bd26-ff52-c59d-2e8d-a7ff154d448a
expire_time         2021-03-13T23:05:46.589912+08:00
explicit_max_ttl    0s
id                  s.mSaXjM2gjtQrbh2h1sRdMktF
issue_time          2021-03-13T22:35:46.589915+08:00
meta                map[role_name:jenkins]
num_uses            0
orphan              true
path                auth/approle/login
policies            [default jenkins-policy]
renewable           true
ttl                 6m14s
type                service


## Response Wrap the SecretID
The RoleID is equivalent to a username, and SecretID is the corresponding password. The app needs both to log in with Vault. Naturally, the next question becomes how to deliver those values to the client securely.

A common solution involves three personas instead of two: admin, app, and trusted entity. The trusted entity delivers the RoleID and SecretID to the client by separate means.

For example, Terraform as a trusted entity can deliver the RoleID onto the virtual machine. When the app runs on the virtual machine, the RoleID already exists on the virtual machine.

<img src="images/vault-approle-workflow2.png">

SecretID is like password.  Use **response wrapping** so that only expecting client can unwrap the SecretID.

In [ ]:
## parameter order matters
vault write -wrap-ttl=60s -f auth/approle/role/myapp/secret-id.

In [ ]:
VAULT_TOKEN=s.ILPSggHj70JSzTWpcWtot7nU vault unwrap

In [ ]:
# should give error
VAULT_TOKEN=s.ILPSggHj70JSzTWpcWtot7nU vault unwrap

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">

---
# AppRole Auhentication Method

Vault provides internal and external authentication methods.  External methods are called *tusted third-party authenticators* such as AWS, LDAP, Github, etc.

In some situations trusted third-party authenticator is not available, so Vault has an alternate approach - **AppRole** .   
AppRole allows machines or apps to authenticate.  This auth method is oriented to automated workflows (machines and services) and is less useful for human operators.

* AppRole generates **RoleID** and **SecretID**
    - RoleID is an identifier similar to username
    - SecretID is a credential to login (similart to password)   
    

### Setup
<img src="images/vault-approle-workflow.png" width=800>


In [ ]:
vault login root
vault auth enable approle

In [ ]:
vault write auth/approle/role/tio-crm \
  token_num_users=3 \
  token_ttl=10m \
  token_max_ttl=30m \
  secret_id_ttl=5m \
  secret_id_num_uses=40 

### RoleID
RoleID is a unique identifier for the app.  They can be set to particular values to match introspected information by the client (ex: client's domain name).

In [ ]:
vault read auth/approle/role/tio-crm/role-id

### SecretID
SecretID is like a password, it is intended to always be secret.  SecretIDs can be created agains an AppRole either via generation of a 128-bit purely random UUID by the role itself or via specific custom values.  SecretIDs have properties like usage-limit, TTLs, and expirations.

**Pull and Push SecretID Modes**  
If SecretID is fetched from an AppRole, this is Pull Mode.  Most cases Pull mode is the better approach.

<img src="images/vault-approle-pull.png" width=800>  

If a "custom" secretID is set by the client, it is Push Mode.  
<img src="images/vault-approle-push.png" width=800>

Push Mode requires some other system to have knowledge of the full set of credentials (RoleID and SecretID) in order to create the entry.   
However in Pull Mode, the SecretID can be kept confidential from all parties except for the final authenticating client by using Response w Wrapping.
  


In [ ]:
vault write -f auth/approle/role/tio-crm/secret-id

### Login

In [ ]:
vault write auth/approle/login \
  role_id=d56c9c46-7198-4238-fb0c-3a2b26e97e18   \
  secret_id=e151cb41-0152-8974-e259-75d9fd7c2972 

**Using cURL**  
The default endpoint is `auth/approle/login`.  The response will contain the token at `auth.client_token`

In [ ]:
curl -s --request POST \
    --data "{\"role_id\":\"d56c9c46-7198-4238-fb0c-3a2b26e97e18\", \"secret_id\":\"e151cb41-0152-8974-e259-75d9fd7c2972\"}" \
    http://127.0.0.1:8200/v1/auth/approle/login |jq